In [6]:
## import the tools
import torch
import glob
from pathlib import Path
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN
import glob
import pandas as pd
import ipdb
import laspy
import open3d as o3d

In [7]:
## import the model tools
from torch_geometric.transforms import Compose
from torch_points3d.core.data_transform import GridSampling3D, MinPoints, XYZFeature, AddFeatsByKeys
from torch_points3d.applications.pretrained_api import PretainedRegistry
from torch_geometric.data import Batch,Dataset, Data ,DataLoader

In [ ]:
from sklearn.neighbors import BallTree, KDTree
import numpy as np

In [8]:
model_folder = "2023-04-10/11-10-04/"
root_path = "/home/frederik/Desktop/torch-3dpoints-powerline/"
pl_models_path = root_path+ "outputs/" + model_folder + "SEUNet18.pt" 
model = torch.load(pl_models_path)
model['run_config']['data']['dataroot'] = '/home/frederik/data'
torch.save(model, root_path+ "outputs/" + model_folder + "SEUNet18.pt")
#print(model['run_config']["data"]["train_transform"])

## transformer
pos_z = [ "pos_z" ]
list_add_to_x = [ True ]
delete_feats = [ True ]
lparams = ['512']
first_subsampling = model['run_config']['data']['first_subsampling']

transform_test = Compose([MinPoints(512),
                     XYZFeature(add_x=False, add_y=False, add_z= True),
                     AddFeatsByKeys(list_add_to_x=list_add_to_x, feat_names= pos_z,delete_feats=delete_feats),
                     GridSampling3D(mode='last', size=first_subsampling, quantize_coords=True)])

In [9]:
model_pl = PretainedRegistry.from_file(pl_models_path).cuda()

loading processed train split
Total of 383 samples in train set.
loading processed val split
Total of 154 samples in val set.
loading processed test split
Total of 464 samples in test set.


In [10]:
## to find the neighbor points prediction

def get_nearest(src_points, candidates, k_neighbors=1):
    """Find nearest neighbors for all source points from a set of candidate points"""
    # src : whole gt points in a file 
    tree = KDTree(candidates, leaf_size=20, metric='euclidean')

    # Find closest points and distances
    distances, indices = tree.query(src_points, k=k_neighbors)

    # Transpose to get distances and indices into arrays
    distances = distances.transpose()
    indices = indices.transpose()

    # Get closest indices and distances (i.e. array at index 0)
    closest = np.squeeze(indices)
    closest_dist = distances
    return closest

In [11]:
## load test pt with normalized
processed_folder_name = model['run_config']['data']["processed_folder"]
data_root_path = "/home/frederik/data/denmark"
processed_data_root_path = os.path.join(data_root_path, processed_folder_name)#"/home/jf/msc_data/denmark/processed_hough"
test_fold = processed_data_root_path + "/test_0_(0.05, 0.05)/"
pre_trans_path = test_fold + "/stats.pt"
pre_transform = torch.load(pre_trans_path)

print(f"{pre_transform=}\n")


# Define global variables
global_z = pre_transform['global_z']
room_names = pre_transform['room_names']
room_coord_mins = pre_transform['room_coord_min']
room_coord_maxs = pre_transform['room_coord_max']
room_coord_scales = pre_transform['room_coord_scale']


for file in room_names:
    
    points_list = []
    preds_list = []
    pt_files = glob.glob(test_fold + file + "_cloud*pt")
    for pt_file in pt_files:
        
        pt_data = torch.load(pt_file)
                
        # Define Local Variables
        points = pt_data['points']
        room_index = pt_data['room_idx']
        room_coord_scale = room_coord_scales[room_index]
        pt_ori = points * room_coord_scale + room_coord_mins[room_index]

        # Transform data corresponding to the 
        data_s = transform_test(Batch(pos=torch.from_numpy(points).float()))
        data_s.batch = torch.zeros(len(data_s.pos))
        data_s.y = torch.zeros(data_s.pos.shape[0]).long()

        # Apply Model
        with torch.no_grad():
            model_pl.eval()
            model_pl.set_input(data_s, "cuda")
            model_pl.forward(data_s)

        # Use scores
        scores = model_pl.output.cpu().numpy()
        probabilities = torch.nn.functional.softmax(torch.tensor(scores), dim=1)
        predictions = np.argmax(probabilities, axis=1)

        # Map indexes from x,y,z to the voxelized data and their predictions
        indexes = get_nearest(points, data_s.pos)

        points_predictions = [predictions[index].numpy() for _, index in enumerate(indexes)]

        preds_list.append(points_predictions)
        points_list.append(pt_ori)

    preds_list = np.array([item for sublist in preds_list for item in sublist])
    points_list = np.array([item for sublist in points_list for item in sublist])
    
    las = laspy.read("/home/frederik/data/denmark/raw/test/LazFilesWithHeightParam/" +file +"_hag_nn.laz", laz_backend=laspy.compression.LazBackend.LazrsParallel)
    points_data = np.stack([las.X, las.Y, las.Z], axis=0).transpose((1, 0))

    las.add_extra_dim(laspy.ExtraBytesParams(
        name="prediction",
        type=np.uint8,
        description="The prediction of the model"
    ))
    
    indexes = get_nearest(points_list, points_data)
    
    
    final_las_preds = np.zeros(len(las.prediction))
    for i, index in enumerate(indexes):
        if preds_list[i]:
            final_las_preds[index] = preds_list[i]
    las.prediction = final_las_preds
    
    powerline_las = las[las.prediction == 1]
    powerline_points_data = np.stack([powerline_las.X, powerline_las.Y, powerline_las.Z], axis=0).transpose((1, 0))
    
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(powerline_points_data)
    o3d.visualization.draw_geometries([geom])
    
    succesfully_predicted_powerline_points = np.sum((las.classification == 14) & (las.prediction == 1))
    percentage = succesfully_predicted_powerline_points/np.sum(las.classification == 14)

    print("Predicted Powerline Points: ", succesfully_predicted_powerline_points)
    print("Powerline Accuracy: ", percentage)

pre_transform={'room_names': ['PUNKTSKY_00005_1km_6162_472', 'PUNKTSKY_00005_1km_6161_465'], 'room_coord_min': [array([4.7200001e+07, 6.1620000e+08, 2.3660000e+03]), array([4.6500001e+07, 6.1610000e+08, 2.3660000e+03])], 'room_coord_max': [array([4.72999920e+07, 6.16299999e+08, 1.20160000e+04]), array([4.65999990e+07, 6.16199999e+08, 1.20160000e+04])], 'room_coord_scale': [array([4999.55, 4999.95, 9650.  ]), array([4999.9 , 4999.95, 9650.  ])], 'global_z': (2366.0, 12016.0)}

4095
Predicted Powerline Points:  66351
Powerline Accuracy:  0.9815091492729398
4095
Predicted Powerline Points:  4684
Powerline Accuracy:  0.6328874476422105


In [12]:
print(np.min(points_list[:,0]))
print(np.max(points_list[:,0]))

print(np.min(points_list[:,1]))
print(np.max(points_list[:,1]))

print(np.min(points_list[:,2]))
print(np.max(points_list[:,2]))

46500001.0
46599999.0
616100000.0
616199999.0
170.0
13080.999999999998


In [13]:
print(np.min(points_data[:,0]))
print(np.max(points_data[:,0]))

print(np.min(points_data[:,1]))
print(np.max(points_data[:,1]))

print(np.min(points_data[:,2]))
print(np.max(points_data[:,2]))

46500000
46600000
616100000
616200000
170
13081
